In [1]:
TRAIN_DF = "train_df.csv"

TEST_DF = "test_df.csv"
BOW_MODELS = "Models/BoW/" 
WE_MODELS = "Models/WE/" 
WORD_EMBEDDINGS = "../word_embeddings/"
PRE_METHOD='Raw' #'Lemmatized' for bow and we, 'Raw' for lm
#WE_MODEL="GLOVE_CC"
LM_MODEL='all-mpnet-base-v2'#'all-MiniLM-L6-v2'
FEATURE_TYPE="NN" #"BoW" #"LM" #"WE"
ROLLING_WINDOW_SIZE=10
SENT=False
ROUND_MAX=100

In [2]:
import pandas as pd
import numpy as np
from utilities import rolling_window, sa_features, save_embeddings
import optuna
from sentence_transformers import SentenceTransformer

import joblib
seed=23
np.random.seed(seed)

train_df = pd.read_csv(TRAIN_DF, sep='\t')
test_df = pd.read_csv(TEST_DF, sep='\t')
#train_df = rolling_window(train_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
#test_df = rolling_window(test_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
full_df = pd.concat([train_df,test_df])
###
#full_df = rolling_window(full_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
###
#tokens_df = pd.read_csv("./tokens.csv", sep='\t')
#total_df = tokens_df.merge(full_df.drop_duplicates(), on=['User','Post_Nr','Raw','Stemmed','Lemmatized','Label' ], 
#                   how='left', indicator=True)
#unused_df = total_df[total_df['_merge'] == 'left_only'].copy()
#unused_df.drop(['_merge'], inplace=True, axis=1)
full_df = rolling_window(full_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
#full_df = full_df.query("Window_id < 90")


#unused_df = rolling_window(unused_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
#unused_df = unused_df.query("Window_id < 90")
"""
"""
"""
lm=SentenceTransformer(LM_MODEL)
lm.max_seq_length=512 
unused_embeddings = lm.encode(unused_df['Text'].tolist(),show_progress_bar=True,\
            output_value='sentence_embedding', batch_size=64,convert_to_numpy=True)
save_embeddings("unused_embeddings.pkl", unused_embeddings)
"""

#if SENT:
#    full_df = sa_features(full_df)
#    unused_df = sa_features(unused_df)
#full_df.to_pickle("full_df_sent.pkl")

#full_df = pd.read_pickle("full_df_sent.pkl")

#full_df.head()

'\nlm=SentenceTransformer(LM_MODEL)\nlm.max_seq_length=512 \nunused_embeddings = lm.encode(unused_df[\'Text\'].tolist(),show_progress_bar=True,            output_value=\'sentence_embedding\', batch_size=64,convert_to_numpy=True)\nsave_embeddings("unused_embeddings.pkl", unused_embeddings)\n'

In [3]:
#unused_df['Vector'] = pd.DataFrame(data=unused_embeddings).values.tolist()
#unused_df

In [4]:
#unused_df.to_pickle("final_unused.pkl")


In [5]:
full_df

,User,Window_id,Text,Label
0,test_subject1345,0,so many unwanted smith fadeaways. mid range j...,1
1,test_subject1345,1,"mid range jumpers hey guys, celtics fan here p...",1
2,test_subject1345,2,well he got number tonight so maybe he will b...,1
3,test_subject1345,3,i mean he will get pinch hits and an occasion...,1
4,test_subject1345,4,yeah you are probably right. oh well. i gues...,1
...,...,...,...,...
210922,subject9832,995,exactly scumbag christian girl.. i find myse...,0
210923,subject9832,996,scumbag christian girl.. i find myself doing ...,0
210924,subject9832,997,i find myself doing this everytime i walk thou...,0
210925,subject9832,998,...snow days keep your children from school.....,0


In [6]:
"""
### BoW FEATURES
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
nltk_stop_words = set(stopwords.words("english"))
import optuna
import joblib
TFIDF_STUDY_PATH="./BagOfWords/t2_tfidfvec_10"
MODEL_STUDY_PATH="./BagOfWords/t2_tuning_10.pkl"
tfidf_study = joblib.load(TFIDF_STUDY_PATH)
model_study = joblib.load(MODEL_STUDY_PATH)
tfidf_params = tfidf_study.best_trial.params
tfidf_params['stop_words'] = nltk_stop_words
tfidf_params['ngram_range'] = (1,tfidf_params['ngram_range'])
vectorizer = TfidfVectorizer(**tfidf_params)
model = MultinomialNB(**model_study.best_params)

"""

'\n### BoW FEATURES\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.linear_model import SGDClassifier\nfrom sklearn.naive_bayes import MultinomialNB\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom nltk.corpus import stopwords\nnltk_stop_words = set(stopwords.words("english"))\nimport optuna\nimport joblib\nTFIDF_STUDY_PATH="./BagOfWords/t2_tfidfvec_10"\nMODEL_STUDY_PATH="./BagOfWords/t2_tuning_10.pkl"\ntfidf_study = joblib.load(TFIDF_STUDY_PATH)\nmodel_study = joblib.load(MODEL_STUDY_PATH)\ntfidf_params = tfidf_study.best_trial.params\ntfidf_params[\'stop_words\'] = nltk_stop_words\ntfidf_params[\'ngram_range\'] = (1,tfidf_params[\'ngram_range\'])\nvectorizer = TfidfVectorizer(**tfidf_params)\nmodel = MultinomialNB(**model_study.best_params)\n\n'

In [7]:
"""
### WE FEATURES
from utilities import WordEmbeddingVectorizer
from sklearn.linear_model import SGDClassifier
import optuna
import joblib
STUDY_PATH="./Embeddings/t2_tuning_10.pkl"
study = joblib.load(STUDY_PATH)
model = SGDClassifier(**study.best_params)
we_vectorizer = WordEmbeddingVectorizer(WE_MODEL, WORD_EMBEDDINGS)
#full_df = full_df.sample(frac=1, random_state=seed).reset_index(drop=True) 
full_df['Vector'] = full_df['Text'].apply(lambda text: we_vectorizer.vectorize(text))
unused_df['Vector'] = unused_df['Text'].apply(lambda text: we_vectorizer.vectorize(text))

full_df
"""

'\n### WE FEATURES\nfrom utilities import WordEmbeddingVectorizer\nfrom sklearn.linear_model import SGDClassifier\nimport optuna\nimport joblib\nSTUDY_PATH="./Embeddings/t2_tuning_10.pkl"\nstudy = joblib.load(STUDY_PATH)\nmodel = SGDClassifier(**study.best_params)\nwe_vectorizer = WordEmbeddingVectorizer(WE_MODEL, WORD_EMBEDDINGS)\n#full_df = full_df.sample(frac=1, random_state=seed).reset_index(drop=True) \nfull_df[\'Vector\'] = full_df[\'Text\'].apply(lambda text: we_vectorizer.vectorize(text))\nunused_df[\'Vector\'] = unused_df[\'Text\'].apply(lambda text: we_vectorizer.vectorize(text))\n\nfull_df\n'

In [8]:
#unused_df.to_pickle("unused_pre.pkl", protocol=4)

In [9]:
### LM FEATURES
"""
from sklearn.linear_model import SGDClassifier
import optuna
import joblib
from utilities import load_embeddings
STUDY_PATH = "./LanguageModels/t2_tuning_10.pkl"
study = joblib.load(STUDY_PATH)
study.best_params
model = SGDClassifier(**study.best_params)
train_sentence_embeddings = load_embeddings(f"./LanguageModels/train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
val_sentence_embeddings = load_embeddings(f"./LanguageModels/val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
#unused_embeddings = load_embeddings(f"./unused_embeddings.pkl")

full_sentence_embeddings = np.concatenate((train_sentence_embeddings,val_sentence_embeddings), axis=0)#full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
full_df = full_df.query("Window_id < 90")

unused_df = pd.read_pickle("final_unused.pkl")

#unused_df = pd.read_pickle("final_unused.pkl")
#full_df = pd.read_pickle("full_df_pkl.pkl")
"""



'\nfrom sklearn.linear_model import SGDClassifier\nimport optuna\nimport joblib\nfrom utilities import load_embeddings\nSTUDY_PATH = "./LanguageModels/t2_tuning_10.pkl"\nstudy = joblib.load(STUDY_PATH)\nstudy.best_params\nmodel = SGDClassifier(**study.best_params)\ntrain_sentence_embeddings = load_embeddings(f"./LanguageModels/train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")\nval_sentence_embeddings = load_embeddings(f"./LanguageModels/val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")\n#unused_embeddings = load_embeddings(f"./unused_embeddings.pkl")\n\nfull_sentence_embeddings = np.concatenate((train_sentence_embeddings,val_sentence_embeddings), axis=0)#full_df[\'Vector\'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()\nfull_df[\'Vector\'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()\nfull_df = full_df.query("Window_id < 90")\n\nunused_df = pd.read_pickle("final_unused.pkl")\n\n#unused_df = pd.read_pickle("final_unused.pkl")\

In [10]:
"""
"""
### LM FEATURES
from sklearn.linear_model import SGDClassifier
import torch
import optuna
import joblib
from utilities import load_embeddings, LmNeuralNetwork, WritingWindowDataset
STUDY_PATH = "./LanguageModels/NN_hyperparameterstudy10.pkl"
study = joblib.load(STUDY_PATH)
study.best_params
params = optuna.trial.FixedTrial(study.best_trial.params)
#model = torch.load()
#model.eval()
train_sentence_embeddings = load_embeddings(f"./LanguageModels/train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
val_sentence_embeddings = load_embeddings(f"./LanguageModels/val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
#unused_embeddings = load_embeddings(f"./unused_embeddings.pkl")

full_sentence_embeddings = np.concatenate((train_sentence_embeddings,val_sentence_embeddings), axis=0)#full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
unused_df = pd.read_pickle("final_unused.pkl")

train_ds = WritingWindowDataset(full_sentence_embeddings, full_df['Label'])
#unused_df = pd.read_pickle("final_unused.pkl")

#full_df = pd.read_pickle("full_df_pkl.pkl")



In [11]:
"""
"""
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import f1_score
#from torchmetrics.functional import f1_score
BATCH_SIZE = 32
epochs=10

def train_nn(trainloader, study):
    params = optuna.trial.FixedTrial(study.best_trial.params)
    model= LmNeuralNetwork(params)

    model = model.cuda()
    optimizer = AdamW(model.parameters(), lr=study.best_trial.params['learning_rate'])#, lr= learning_rate)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()

            

    for epoch_num in range(epochs):
        
        total_loss_train = 0
        #train_preds = torch.tensor([], requires_grad=False).to(device)
        #train_targets = torch.tensor([], requires_grad=False).to(device)
        train_preds = np.array([])
        train_targets = np.array([])
        model.train()
        for train_input, train_label in tqdm(trainloader):

            train_label = train_label.to(device)
            train_label = train_label.float()
            train_label = train_label.unsqueeze(1)
            features = train_input.to(device)
            

            output = model(features)
            batch_loss = criterion(output, train_label)
            total_loss_train += float(batch_loss.item())
            
            #train_preds = torch.cat((train_preds,torch.round(output).detach()),0)
            #train_targets=torch.cat((train_targets,train_label.detach()),0)
            train_preds = np.concatenate((train_preds,torch.round(output).detach().cpu().numpy().flatten()),axis=0)
            train_targets = np.concatenate((train_targets,train_label.detach().cpu().numpy().flatten()),axis=0)

            
            #acc = (train_y_pred_tag == train_label).sum().item()
            #total_acc_train += float(acc)
            #seen_train+=len(train_preds)
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        train_loss=total_loss_train/len(train_preds)
        train_f1 = f1_score(train_targets,train_preds)
            
        print(
                f'Epochs: {epoch_num + 1} | Train Loss: {train_loss: .4f} \
                | Train F1: {train_f1: .4f}')

        
    return model

In [12]:
"""

from sklearn.model_selection import StratifiedKFold

def cv_classification(full_df, model,filename):
    skf = StratifiedKFold(n_splits=5)
    user_label_df =full_df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    labels = user_label_df['Label'].to_numpy()
    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):
            train_users = [users[f] for f in train_index]
            test_users = [users[f] for f in test_index]

            train_folds = full_df[full_df['User'].isin(train_users)].copy()
            train_folds = train_folds.sample(frac=1, random_state=seed).reset_index(drop=True) 

            test_folds = full_df[full_df['User'].isin(test_users)].copy()
            test_folds = pd.concat([test_folds,unused_df])

            if FEATURE_TYPE=="BoW":
                X_train = vectorizer.fit_transform(train_folds['Text'])
            else:
                X_train = pd.DataFrame(train_folds['Vector'].values.tolist(), index = train_folds.index)
            y_train = train_folds['Label']
            if SENT:
                X_train = np.c_[X_train,train_folds['polarity'],train_folds['subjectivity'],train_folds['negativity'],train_folds['positivity'],train_folds['neutrality'], train_folds['compound']] 

            model.fit(X_train, y_train)
            print(f"model for fold {fold} trained")
            true_label=test_folds[['User','Label']].groupby('User').max().to_dict()['Label']
            true_labels = {subject:true_label[subject] for subject in true_label}
            with open(f"{filename}_{fold}.txt", "w") as f:
                for user in true_labels:
                    f.write(f"({user},{true_labels[user]})")
                    for window_id in range(ROUND_MAX-ROLLING_WINDOW_SIZE+1):
                        test_f = test_folds.query("User==@user and Window_id==@window_id")
                        if len(test_f)>0:
                            f.write(" ")
                            if FEATURE_TYPE=="BoW":
                                X_test = vectorizer.transform(test_f['Text'])
                                if SENT:
                                    X_test = np.r_[X_test,test_f['polarity'],test_f['subjectivity'],test_f['negativity'],test_f['positivity'],test_f['neutrality'], test_f['compound']] 
                                X_test.reshape(1,-1)
                            else:
                                X_test = test_f['Vector'].values.tolist()[0]
                                if SENT:
                                    X_test = np.r_[X_test,test_f['polarity'],test_f['subjectivity'],test_f['negativity'],test_f['positivity'],test_f['neutrality'], test_f['compound']] 
                                X_test = [X_test] #added

                            
                            pred = model.predict(X_test)[0]
                            prob = model.predict_proba(X_test)[0].tolist()[1]
                            f.write(f"({pred},{prob})")
                    f.write("\n")
"""

'\n\nfrom sklearn.model_selection import StratifiedKFold\n\ndef cv_classification(full_df, model,filename):\n    skf = StratifiedKFold(n_splits=5)\n    user_label_df =full_df.drop_duplicates(\'User\')\n    users = user_label_df[\'User\'].to_numpy()\n    labels = user_label_df[\'Label\'].to_numpy()\n    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):\n            train_users = [users[f] for f in train_index]\n            test_users = [users[f] for f in test_index]\n\n            train_folds = full_df[full_df[\'User\'].isin(train_users)].copy()\n            train_folds = train_folds.sample(frac=1, random_state=seed).reset_index(drop=True) \n\n            test_folds = full_df[full_df[\'User\'].isin(test_users)].copy()\n            test_folds = pd.concat([test_folds,unused_df])\n\n            if FEATURE_TYPE=="BoW":\n                X_train = vectorizer.fit_transform(train_folds[\'Text\'])\n            else:\n                X_train = pd.DataFrame(train_folds[\'V

In [13]:
####LOAD MODEL

torch.manual_seed(23)

In [14]:

from sklearn.model_selection import StratifiedKFold

def nn_cv_classification(full_df, study,filename):
    skf = StratifiedKFold(n_splits=5)
    user_label_df =full_df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    labels = user_label_df['Label'].to_numpy()
    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):
            train_users = [users[f] for f in train_index]
            test_users = [users[f] for f in test_index]
            train_ids = train_df.index[train_df['User'].isin(train_users)].to_list()
            train_folds = full_df[full_df['User'].isin(train_users)].copy()
            train_folds = train_folds.sample(frac=1, random_state=seed).reset_index(drop=True) 

            test_folds = full_df[full_df['User'].isin(test_users)].copy()
            test_folds = pd.concat([test_folds,unused_df])
            #X_train = pd.DataFrame(train_folds['Vector'].values.tolist(), index = train_folds.index)
            #y_train = train_folds['Label']
            train_data = torch.utils.data.Subset(train_ds, train_ids)
            
            trainloader = torch.utils.data.DataLoader(
                            train_data, 
                            batch_size=32, 
                            shuffle=True)
            model = train_nn(trainloader, study)
            model.eval()
            print(f"model for fold {fold} trained")
            true_label=test_folds[['User','Label']].groupby('User').max().to_dict()['Label']
            true_labels = {subject:true_label[subject] for subject in true_label}
            with open(f"{filename}_{fold}.txt", "w") as f:
                for user in true_labels:
                    f.write(f"({user},{true_labels[user]})")
                    for window_id in range(ROUND_MAX-ROLLING_WINDOW_SIZE+1):
                        test_f = test_folds.query("User==@user and Window_id==@window_id")
                        if len(test_f)>0:
                            f.write(" ")
                            
                            X_test = test_f['Vector'].values.tolist()[0]
                            with torch.no_grad():
                                X_test = torch.tensor([X_test]).cuda() #added

                                
                                pred = model(X_test)[0]
                                f.write(f"({round(pred.item())},{pred.item()})")
                    f.write("\n")


In [15]:
nn_cv_classification(full_df,study, "NN")
#cv_classification(full_df,model, FEATURE_TYPE)



100%|██████████| 4200/4200 [00:58<00:00, 71.86it/s] 


Epochs: 1 | Train Loss:  0.0212                 | Train F1:  0.0350


100%|██████████| 4200/4200 [00:14<00:00, 286.72it/s]


Epochs: 2 | Train Loss:  0.0204                 | Train F1:  0.3512


100%|██████████| 4200/4200 [00:14<00:00, 286.95it/s]


...

Epochs: 8 | Train Loss:  0.0166                 | Train F1:  0.6546


100%|██████████| 5443/5443 [00:19<00:00, 273.98it/s]


Epochs: 9 | Train Loss:  0.0164                 | Train F1:  0.6589


100%|██████████| 5443/5443 [00:19<00:00, 276.50it/s]


Epochs: 10 | Train Loss:  0.0164                 | Train F1:  0.6574
model for fold 4 trained


In [10]:
import ast
from sklearn.metrics import f1_score
def criterion_evaluation(filename, max_round, criterion, threshold, conf_threshold=0):
    f1_scores = []
    for fold in range(5):
        with open(f"{filename}_{fold}.txt", "r") as f:
            true_labels = []
            classifications = []
            for line in f:
                splt = line[:-1].split(" ")
                splt = [ast.literal_eval(t) for t in splt]
                user, label = splt[0]
                true_labels.append(label)
                classification = 0
                count = 0
                for i, (pred, conf) in enumerate(splt[1:]):
                    if (i + 10)>max_round:
                        break
                    if criterion == "confidence":#  and i%10==0:
                        if pred==1 and conf >= threshold :
                            classification= 1
                    if criterion == "ratio" and conf >=conf_threshold:# and i%10==0:
                        count = count + pred
                        if count/(10 + i) >= threshold:
                            classification=1
                    if criterion == "consecutive":# and i%10==0: ##added stride
                        if pred==1 and conf >=conf_threshold:
                            count = count + 1
                        else:
                            count = 0
                        if count == threshold:
                            classification = 1
                    
                classifications.append(classification)
        f1_scores.append(f1_score(true_labels,classifications))
    return f1_scores

In [11]:
max_round=100

In [12]:
def objective_confidence(trial):
    params = {
        "threshold": trial.suggest_float("threshold", 0.5, 1)
    }
    return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"confidence", params["threshold"]))

In [13]:
def objective_ratio(trial):
    params = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.75)
    }
    return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"ratio", params["threshold"],conf_threshold=0.994))


In [14]:
def objective_consecutive(trial):
    params = {
        "threshold": trial.suggest_int("threshold", 2,max_round-10)
    }
    return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"consecutive", params["threshold"], conf_threshold=0.994))


In [15]:
study = optuna.create_study(study_name=f"{FEATURE_TYPE}_confidence",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
#study = joblib.load(f"{FEATURE_TYPE}_confidence.pkl")

study.optimize(objective_confidence, n_trials=100, timeout=(60*60*3), gc_after_trial=True)
joblib.dump(study, f"{FEATURE_TYPE}_confidence_{max_round}.pkl")
study.best_trial.params, study.best_value

[I 2022-06-21 16:54:13,659] A new study created in memory with name: BoW_confidence
[W 2022-06-21 16:54:13,697] Trial 0 failed because of the following error: ValueError('malformed node or string: <ast.Name object at 0x000001E9AC2BEE50>')
Traceback (most recent call last):
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\rodri\AppData\Local\Temp\ipykernel_11920\1469894624.py", line 5, in objective_confidence
    return np.mean(criterion_evaluation(FEATURE_TYPE,max_round,"confidence", params["threshold"]))
  File "C:\Users\rodri\AppData\Local\Temp\ipykernel_11920\706824801.py", line 11, in criterion_evaluation
    splt = [ast.literal_eval(t) for t in splt]
  File "C:\Users\rodri\AppData\Local\Temp\ipykernel_11920\706824801.py", line 11, in <listcomp>
    splt = [ast.literal_eval(t) for t in splt]
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python

ValueError: malformed node or string: <ast.Name object at 0x000001E9AC2BEE50>

In [24]:
study2 = optuna.create_study(study_name=f"{FEATURE_TYPE}_ratio",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
#study = joblib.load(f"{FEATURE_TYPE}_ratio.pkl")

study2.optimize(objective_ratio, n_trials=100, timeout=(60*60*3), gc_after_trial=True)
#joblib.dump(study2, f"{FEATURE_TYPE}_ratio_{max_round}.pkl")
study2.best_trial.params, study2.best_value

[I 2022-06-21 07:23:00,724] A new study created in memory with name: WE_ratio
[I 2022-06-21 07:23:22,650] Trial 0 finished with value: 0.2519087830691231 and parameters: {'threshold': 0.295431289521223}. Best is trial 0 with value: 0.2519087830691231.
[I 2022-06-21 07:23:44,588] Trial 1 finished with value: 0.2903782519060936 and parameters: {'threshold': 0.23027026565814124}. Best is trial 1 with value: 0.2903782519060936.
[I 2022-06-21 07:24:06,433] Trial 2 finished with value: 0.35498099906841113 and parameters: {'threshold': 0.08190665485629145}. Best is trial 2 with value: 0.35498099906841113.
[I 2022-06-21 07:24:28,332] Trial 3 finished with value: 0.12543880025266418 and parameters: {'threshold': 0.5250840141245}. Best is trial 2 with value: 0.35498099906841113.
[I 2022-06-21 07:24:50,569] Trial 4 finished with value: 0.19675147547487973 and parameters: {'threshold': 0.36693676876019476}. Best is trial 2 with value: 0.35498099906841113.
[I 2022-06-21 07:25:12,605] Trial 5 finish

({'threshold': 0.052775584714150454}, 0.37953098848650046)

In [25]:
study3 = optuna.create_study(study_name=f"{FEATURE_TYPE}_consecutive",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
study3.optimize(objective_consecutive, n_trials=100, timeout=(60*60*3), gc_after_trial=True)
#joblib.dump(study3, f"{FEATURE_TYPE}_consecutive_{max_round}.pkl")
study3.best_trial.params, study3.best_value

[I 2022-06-21 07:59:28,113] A new study created in memory with name: WE_consecutive
[I 2022-06-21 07:59:49,863] Trial 0 finished with value: 0.01111111111111111 and parameters: {'threshold': 71}. Best is trial 0 with value: 0.01111111111111111.
[I 2022-06-21 08:00:11,758] Trial 1 finished with value: 0.05263157894736842 and parameters: {'threshold': 29}. Best is trial 1 with value: 0.05263157894736842.
[I 2022-06-21 08:00:33,662] Trial 2 finished with value: 0.01111111111111111 and parameters: {'threshold': 65}. Best is trial 1 with value: 0.05263157894736842.
[I 2022-06-21 08:00:55,578] Trial 3 finished with value: 0.01142857142857143 and parameters: {'threshold': 78}. Best is trial 1 with value: 0.05263157894736842.
[I 2022-06-21 08:01:17,486] Trial 4 finished with value: 0.01111111111111111 and parameters: {'threshold': 71}. Best is trial 1 with value: 0.05263157894736842.
[I 2022-06-21 08:01:39,416] Trial 5 finished with value: 0.01111111111111111 and parameters: {'threshold': 56}.

({'threshold': 2}, 0.39021134796010515)

In [18]:
"""
class T1Classifier:
    def __init__(self, name,  model, textual_features, criterion, sentiment=False,window=5):
        self.name=name
        self.window=window
        self.textual_features=textual_features
        self.model=model
        self.sentiment=sentiment
        self.criterion=criterion
        if self.textual_features=='tfidf':
            self.feature_extractor = pickle.load(open(f"{BOW_MODELS}TFIDF_win_{self.window}/tfidf_vectorizer_{self.window}",'rb'))
        elif self.textual_features =='glove':
            self.feature_extractor = glove_embeddings
        elif self.textual_features=='sbert':
            self.feature_extractor=lm_model

        self.decision_map={}



    def preprocess(self,text):
        if self.textual_features!='lm':
            text=simplify(text, method='lem')
        
        text_features=None
        
        if self.textual_features=='tfidf':
            text_features=self.feature_extractor.transform([text])
        elif self.textual_features=='glove':
            text_features = get_document_embeddings(text,self.feature_extractor)
        elif self.textual_features=='sbert':
            text_features = self.feature_extractor.encode(text,output_value='sentence_embedding', convert_to_numpy=True, batch_size=64)
        
        if self.sentiment:
            tb_5 = TextBlob(text).sentiment
            pol,subj = tb_5
            vader_5 =sia.polarity_scores(text)
            neg, pos, neut, compound = vader_5['neg'],vader_5['neu'],vader_5['pos'],vader_5['compound']
            text_features = np.r_[text_features,pol,subj,neg, pos, neut, compound]

        return text_features 

    def predict(self,user, text):
        if not user in self.decision_map:
            self.decision_map[user]=[]
        
        text_features = self.preprocess(text)
        pred = self.model.predict(text_features)
        return pred
        ###criterion



"""

'\nclass T1Classifier:\n    def __init__(self, name,  model, textual_features, criterion, sentiment=False,window=5):\n        self.name=name\n        self.window=window\n        self.textual_features=textual_features\n        self.model=model\n        self.sentiment=sentiment\n        self.criterion=criterion\n        if self.textual_features==\'tfidf\':\n            self.feature_extractor = pickle.load(open(f"{BOW_MODELS}TFIDF_win_{self.window}/tfidf_vectorizer_{self.window}",\'rb\'))\n        elif self.textual_features ==\'glove\':\n            self.feature_extractor = glove_embeddings\n        elif self.textual_features==\'sbert\':\n            self.feature_extractor=lm_model\n\n        self.decision_map={}\n\n\n\n    def preprocess(self,text):\n        if self.textual_features!=\'lm\':\n            text=simplify(text, method=\'lem\')\n        \n        text_features=None\n        \n        if self.textual_features==\'tfidf\':\n            text_features=self.feature_extractor.transf